<a href="https://colab.research.google.com/github/Wadaken43/DeepLearning/blob/main/yolov3_sanko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#google driveにマウント
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/Colab Notebooks/deeplearning

/content/gdrive/MyDrive/Colab Notebooks/deeplearning


モデルのアーキテクチャ

In [ ]:
from torch.nn.modules.batchnorm import BatchNorm2d
import torch
import torch.nn as nn
import torch.nn.functional as F

class YOLOv3(nn.Module):
 def __init__(self,class_n = 80):
   super(YOLOv3 , self).__init__()
   self.class_n = class_n
   self.first_block = nn.Sequential(
                                     nn.Conv2d(3 , 32 , 3 , 1 , 1),
                                     nn.BatchNorm2d(32),
                                     nn.LeakyReLU(),
                                     nn.Conv2d(32 , 64 , 3 , 2 , 1),
                                     nn.BatchNorm2d(64),
                                     nn.LeakyReLU(),
                                   )
   self.residual_block_1 = self.MakeResidualBlock(64)
   self.conv_1 = nn.Conv2d(64 , 128  , 3 , 2 , 1)

   self.residual_block_2 = nn.Sequential(self.MakeResidualBlock(128),self.MakeResidualBlock(128))
   self.conv_2 = nn.Conv2d(128 , 256  , 3 , 2 , 1)
       
   self.residual_block_3 = nn.Sequential(self.MakeResidualBlock(256),self.MakeResidualBlock(256),self.MakeResidualBlock(256),self.MakeResidualBlock(256),self.MakeResidualBlock(256),self.MakeResidualBlock(256),self.MakeResidualBlock(256),self.MakeResidualBlock(256))
   self.conv_3 = nn.Conv2d(256 , 512  , 3 , 2 , 1)

   self.residual_block_4 = nn.Sequential(self.MakeResidualBlock(512),self.MakeResidualBlock(512),self.MakeResidualBlock(512),self.MakeResidualBlock(512),self.MakeResidualBlock(512),self.MakeResidualBlock(512),self.MakeResidualBlock(512),self.MakeResidualBlock(512))
   self.conv_4 = nn.Conv2d(512 , 1024  , 3 , 2 , 1)
   
   self.residual_block_5 = nn.Sequential(self.MakeResidualBlock(1024),self.MakeResidualBlock(1024),self.MakeResidualBlock(1024),self.MakeResidualBlock(1024),)
   self.conv_block = nn.Sequential(self.MakeResidualBlock(1024),self.MakeResidualBlock(1024),self.MakeResidualBlock(1024),self.MakeResidualBlock(1024),)
   
   
   self.scale3_output = nn.Conv2d(1024 , (3 * (4 + 1 +self.class_n)) , 1 , 1 )
   self.scale2_upsample = nn.Conv2d(1024 , 256 , 1 , 1 )

   self.scale2_convblock = nn.Sequential(nn.Sequential(nn.Conv2d(768 , 256 , 1 , 1), 
                          nn.BatchNorm2d(256),
                          nn.LeakyReLU(),
                          nn.Conv2d(256 , 512 , 3 , 1 ,  1),
                          nn.BatchNorm2d(512),
                          nn.LeakyReLU(),
                         ),self.MakeResidualBlock(512),self.MakeResidualBlock(512),)
   
   self.scale2_output = nn.Conv2d(512 , (3 * (4 + 1 +self.class_n)) , 1 , 1 )
   
   self.scale1_upsample = nn.Conv2d(512 , 128 , 1 , 1 )

   self.scale1_convblock = nn.Sequential(nn.Sequential(nn.Conv2d(384 , 128 , 1 , 1), 
                          nn.BatchNorm2d(128),
                          nn.LeakyReLU(),
                          nn.Conv2d(128 , 256 , 3 , 1 ,  1),
                          nn.BatchNorm2d(256),
                          nn.LeakyReLU(),
                         ),self.MakeResidualBlock(256),self.MakeResidualBlock(256),)

   self.scale1_output =  nn.Conv2d(256 , (3 * (4 + 1 +self.class_n)) , 1 , 1 )

   self.upsample = nn.Upsample(scale_factor = 2)

 def MakeResidualBlock(self,fn):
   block = nn.Sequential(nn.Conv2d(fn , int(fn/2) , 1 , 1), 
                          nn.BatchNorm2d(int(fn/2)),
                          nn.LeakyReLU(),
                          nn.Conv2d(int(fn/2) , fn , 3 , 1 ,  1),
                          nn.BatchNorm2d(fn),
                          nn.LeakyReLU(),
                         )
   return block
 
 def forward(self,x):
   x = self.first_block(x)
   x_res = self.residual_block_1(x)
   x = x + x_res
   x = self.conv_1(x)

   for layer in self.residual_block_2:
     x_res = layer(x)
     x = x + x_res
   x = self.conv_2(x)
   
   for layer in self.residual_block_3:
     x_res = layer(x)
     x = x + x_res
   x1 = x
   x = self.conv_3(x)

   for layer in self.residual_block_4:
     x_res = layer(x)
     x = x + x_res
   x2 = x
   x = self.conv_4(x)
   
   for layer in self.residual_block_5:
     x_res = layer(x)
     x = x + x_res
   
   for layer in self.conv_block:
     x = layer(x)
   
   scale3_result = self.scale3_output(x)
   
   scale2_up = self.upsample(self.scale2_upsample(x))
   x = torch.cat([x2 , scale2_up],dim = 1)

   for layer in self.scale2_convblock :
     x = layer(x)

   x2 = x
   scale2_result = self.scale2_output(x)

   scale1_up = self.upsample(self.scale1_upsample(x2))

   x = torch.cat([x1 , scale1_up],dim = 1)
   for layer in self.scale1_convblock :
     x = layer(x)
   scale1_result = self.scale1_output(x)

   return  scale3_result , scale2_result , scale1_result 

In [ ]:
model = YOLOv3()
with torch.no_grad():
  output = model(torch.zeros((1,3,416,416)))
for i in range(3):
  print(output[i].shape)

#torch.Size([1, 255, 13, 13])
#torch.Size([1, 255, 26, 26])
#torch.Size([1, 255, 52, 52])

torch.Size([1, 255, 13, 13])
torch.Size([1, 255, 26, 26])
torch.Size([1, 255, 52, 52])
